# SSC300: A fast model to count multiple classes

In this notebook it is shown a comparison of the serveral variants of the Keras implementation of the SSC model. The notebook can be ran on Google Colab to have a free Tesla K80 12GB GPU.

## Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Download of datasets and Packages decompression
This series of cells allows to download and uncompress the PASCAL VOC 2007 and 2012 

In [0]:
!rm -r object_counting object_classification utils

In [0]:
!tar -xf keras.tar.xz

In [0]:
!rm -r weights

In [0]:
!mkdir weights

In [0]:
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar

In [0]:
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar

In [0]:
!wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar

In [0]:
!tar -xf VOCtest_06-Nov-2007.tar

In [0]:
!tar -xf VOCtrainval_06-Nov-2007.tar

In [0]:
!tar -xf VOCtrainval_11-May-2012.tar

In [0]:
!python -m  pip install tabulate

## Models applied to PASCAL VOC dataset

From now on, I will show you how to use the code and instantiate the different models and test them on the PASCAL VOC dataset.

In each block, most of the operations are the same thanks to the flexibility of the class inspired by Object Oriented paradigm, exploiting objects ineritance and interface style.

### Preliminary imports

In [0]:
from keras.optimizers import Adam, SGD
from keras.losses import poisson
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras.callbacks import TensorBoard
from keras import backend as K
from keras.models import load_model
from math import ceil
import h5py
import numpy as np
import shutil
import time
from object_counting.keras_ssc.misc_utils.tensor_sampling_utils import sample_tensors
from matplotlib import pyplot as plt

from object_counting.keras_ssc.models.keras_ssc300_4_merged import ssc_300 as ssc_300_m
from object_counting.keras_ssc.models.keras_ssc300 import ssc_300 as ssc_300_normal
from object_counting.keras_ssc.keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from object_counting.keras_ssc.keras_layers.keras_layer_L2Normalization import L2Normalization

from object_counting.keras_ssc.ssd_encoder_decoder.ssc_input_encoder import SSCInputEncoder

from object_counting.keras_ssc.data_generator.object_detection_2d_data_generator import DataGenerator
from object_counting.keras_ssc.data_generator.object_detection_2d_geometric_ops import Resize
from object_counting.keras_ssc.data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from object_counting.keras_ssc.data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from object_counting.keras_ssc.data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

### Setting variables

For more info about the variables check the doumentation of the package. If it is not build, you can use sphynx to generate the html docs from the code doc strings.

In [0]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 20 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_pascal = [0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
aspect_ratios = [[1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]]
hidden_sizes = [250, 250, 100]
predictors = ['conv6_2', 'conv7_2', 'conv8_2', 'conv9_2']
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
steps = [32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
normalize_coords = True

## MODEL

### Implement Models
After having defined all the parameters, we instantiate the model, instantiate an optimizer, maybe load weights and compile the model.

In [0]:
K.clear_session() # Clear previous models from memory

model_4 = ssc_300_normal(image_size=(img_height, img_width, img_channels),
                         n_classes=n_classes,
                         mode='training',
                         l2_regularization=0.0005,
                         scales=scales,
                         aspect_ratios_per_layer=aspect_ratios,
                         two_boxes_for_ar1=two_boxes_for_ar1,
                         steps=steps,
                         offsets=offsets,
                         variances=variances,
                         subtract_mean=mean_color,
                         swap_channels=swap_channels,
                         predictors=predictors,
                         output_activation=True,
                         lstm=False,
                         condense_predictors=False)

model_m = ssc_300_m(image_size=(img_height, img_width, img_channels),
                    n_classes=n_classes,
                    mode='training',
                    l2_regularization=0.0005,
                    scales=scales,
                    aspect_ratios_per_layer=aspect_ratios,
                    two_boxes_for_ar1=two_boxes_for_ar1,
                    steps=steps,
                    offsets=offsets,
                    variances=variances,
                    subtract_mean=mean_color,
                    swap_channels=swap_channels,
                    output_activation=True,
                    lstm=False,
                    condense_predictors=False)

model_m_c = ssc_300_m(image_size=(img_height, img_width, img_channels),
                      n_classes=n_classes,
                      mode='training',
                      l2_regularization=0.0005,
                      scales=scales,
                      aspect_ratios_per_layer=aspect_ratios,
                      two_boxes_for_ar1=two_boxes_for_ar1,
                      steps=steps,
                      offsets=offsets,
                      variances=variances,
                      subtract_mean=mean_color,
                      swap_channels=swap_channels,
                      output_activation=True,
                      lstm=False,
                      condense_predictors=True)

model_m_lstmc = ssc_300_m(image_size=(img_height, img_width, img_channels),
                          n_classes=n_classes,
                          mode='training',
                          l2_regularization=0.0005,
                          scales=scales,
                          aspect_ratios_per_layer=aspect_ratios,
                          two_boxes_for_ar1=two_boxes_for_ar1,
                          steps=steps,
                          offsets=offsets,
                          variances=variances,
                          subtract_mean=mean_color,
                          swap_channels=swap_channels,
                          output_activation=True,
                          lstm=True,
                          condense_predictors=True)

# 2: Load some weights into the model.
# TODO: Set the path to the weights you want to load.


# If by_name is set to False, the structure must be identical,
# while if set to True, only the layers with the same name must be identical
print("LOADING WEIGHTS...\n")
model_4.load_weights('gdrive/My Drive/ColabFiles/ssc300_basic_VOC_07+12.h5')
model_m.load_weights('gdrive/My Drive/ColabFiles/ssc300_m_VOC_07+12.h5')
model_m_c.load_weights('gdrive/My Drive/ColabFiles/ssc300_m_c_VOC_07+12.h5')
model_m_lstmc.load_weights('gdrive/My Drive/ColabFiles/ssc300_m_lstm_c_ft_VOC_07+12.h5')

models = [model_4, model_m, model_m_c, model_m_lstmc]

### Load datasets and convert them in the correct format for the model

In [0]:
# 1: Instantiate `DataGenerator` objects: One for training, one for validation.
# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
test_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets. This can take a while.
# TODO: Set the paths to the datasets here.

# The directories that contain the images.
images_dir      = '/content/VOCdevkit/VOC2007/JPEGImages/'
images_dir_12   = '/content/VOCdevkit/VOC2012/JPEGImages/'

# The directories that contain the annotations.
annotations_dir      = '/content/VOCdevkit/VOC2007/Annotations/'
annotations_dir_12   = '/content/VOCdevkit/VOC2012/Annotations/'

# The paths to the image sets.
train_image_set_filename    = '/content/VOCdevkit/VOC2007/ImageSets/Main/train.txt'
train_image_set_filename_12    = '/content/VOCdevkit/VOC2012/ImageSets/Main/trainval.txt'
val_image_set_filename      = '/content/VOCdevkit/VOC2007/ImageSets/Main/val.txt'
test_image_set_filename     = '/content/VOCdevkit/VOC2007/ImageSets/Main/test.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

train_dataset.parse_xml(images_dirs=[images_dir, images_dir_12],
                        image_set_filenames=[train_image_set_filename, train_image_set_filename_12],
                        annotations_dirs=[annotations_dir, annotations_dir_12],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)

val_dataset.parse_xml(images_dirs=[images_dir],
                      image_set_filenames=[val_image_set_filename],
                      annotations_dirs=[annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=True,
                      ret=False)

test_dataset.parse_xml(images_dirs=[images_dir],
                      image_set_filenames=[test_image_set_filename],
                      annotations_dirs=[annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=True,
                      ret=False)

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you want 
# to create HDF5 datasets, uncomment the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='/content/VOCdevkit/VOC2007/voc_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='/content/VOCdevkit/VOC2007/voc_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# test_dataset.create_hdf5_dataset(file_path='/content/VOCdevkit/VOC2007/voc_test.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

In [0]:
# 3: Set the batch size.

batch_size = 1 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.

# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [# model.get_layer('conv4_3_norm').output_shape[1:3],
                   # model.get_layer('fc7').output_shape[1:3],
                   model_4.get_layer('conv6_2').output_shape[1:3],
                   model_4.get_layer('conv7_2').output_shape[1:3],
                   model_4.get_layer('conv8_2').output_shape[1:3],
                   model_4.get_layer('conv9_2').output_shape[1:3]]

ssd_input_encoder = SSCInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

from object_counting.keras_ssc.ssd_encoder_decoder.ssc_input_encoder_1pred import SSCInputEncoder1Pred

ssc_input_encoder = SSCInputEncoder1Pred(len(classes))

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[convert_to_3_channels,
                                                          resize],
                                         label_encoder=ssc_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssc_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)
test_generator = test_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssc_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()
test_dataset_size   = test_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))
print("Number of images in the test dataset:\t\t{:>6}".format(test_dataset_size))

### Evaluate the model

In [0]:
from tqdm import tqdm
from time import time

predictions = []
gts = []
imgs = []
labels = []
times = np.zeros(len(models))

for m in models:
    predictions.append(list())

for i in tqdm(range(ceil(test_dataset_size/batch_size))):
    batch_x, batch_y = next(test_generator)
    imgs.extend(batch_x)
    gts.extend(batch_y)
    
    for j, m in enumerate(models):
        time_temp = time()
        predictions[j].extend(m.predict(batch_x, batch_size))
        times[j] += time() - time_temp

imgs = np.array(imgs)

for i in range(len(predictions)):
    predictions[i]= np.array(predictions[i])
    if i < 2:
        predictions[i] = np.reshape(predictions[i], (-1,len(classes),len(predictor_sizes)), order='F').sum(axis=2)

In [0]:
predictions = np.array(predictions)
gts = np.array(gts)
predictions.shape

#### Metrics of the 4 models
Here you can see the metrics computed per model

In [0]:
from matplotlib.pyplot import imshow
import random
from tabulate import tabulate

def pretty_prediction_multimodel(classes, y_pred, y_true, pred_idx=0):
    headers = ['class', 'g_truth', 'basic 4', 'merged', 'm_cond', 'm_lstm_c']
    table = []
    
    for i in range(y_pred.shape[1]):
        print("Prediction {}".format(i))
        for c in range(len(classes)):
            if not np.round(y_pred[:,i,c]).sum() == 0 or not y_true[i,c] == 0:
                row = [classes[c], y_true[i,c]]
                for m in range(y_pred.shape[0]):
                    row.append(np.round(y_pred[m, i, c]))
            table.append(row)  
    print(tabulate(table, headers=headers))

In [0]:
model_names = ['ssc-basic', 'ssc-m', 'ssc-m-c', 'ssc-m-lstm-c']
headers_rmse = ['RMSE', 'mRMSE', 'm_relRMSE']
table_rmse = []

for i, p in enumerate(predictions):
    RMSE = np.sqrt(np.mean((p-gts)**2))
    mRMSE = np.mean(np.sqrt(np.mean((p-gts)**2, axis=0)))
    m_relRMSE = np.mean(np.sqrt(np.mean(((p-gts)**2)/(gts+1), axis=0)))
    table_rmse.append([model_names[i], RMSE, mRMSE, m_relRMSE])

print(tabulate(table_rmse, headers=headers_rmse))
  


#### Some nice plots
The following generated plots represent the error of the models in relation to the count, and subsequently, the percentage of correctly, overly and underly estimated counts per number of instances.

In [0]:
gts_totnum = gts.sum(axis=1)

model_names = ['ssc-basic', 'ssc-m', 'ssc-m-c', 'ssc-m-lstm-c']
headers_num = ['Model', '# Objects', 'mRMSE']
table_num = []

for i in range(15):
    mask = gts_totnum == i + 1
    for j, p in enumerate(predictions):
        mRMSE = np.mean(np.sqrt(np.mean((p[mask,:]-gts[mask, :])**2, axis=0)))
        table_num.append([model_names[j], i + 1, mRMSE])

In [0]:
mRMSE_num = []

for i in range(15):
    a = []
    for j in range(predictions.shape[0]):
        a.append(table_num[i*4+j][2])
        mRMSE_num.append(a)

    mRMSE_num = np.array(mRMSE_num).T
    mRMSE_num = {model_names[i]:mRMSE_num[i] for i in range(mRMSE_num.shape[0])}
    mRMSE_num['x'] = range(1,16)

In [0]:
colors = ['coral', 'goldenrod', 'mediumseagreen', 'steelblue']
plt.rcParams["figure.figsize"] = [8.0, 5.0]
for i in range(predictions.shape[0]):
    plt.plot( 'x', model_names[i], data=mRMSE_num, marker='o', color=colors[i], linewidth=2, label=model_names[i])
plt.legend()
plt.xticks(mRMSE_num['x'])
plt.ylabel('Count Error (mRMSE)')
plt.xlabel('Counts')
plt.grid(True)

In [0]:
gts_totnum_c = gts

model_names = ['ssc-basic', 'ssc-m', 'ssc-m-c', 'ssc-m-lstm-c']
table_num_c = []

for i in range(16):
    mRMSE = np.zeros((predictions.shape[0], len(classes)))
    for j, p in enumerate(predictions):
        for c in range(len(classes)):
            mask = gts_totnum_c[:, c] == i
            if (p[mask,c]-gts[mask, c]).shape[0] == 0:
                continue
            else:
                mRMSE[j, c] += np.sqrt(np.mean((p[mask,c]-gts[mask, c])**2))
            table_num_c.append([model_names[j], i + 1, np.true_divide(mRMSE[j].sum(),(mRMSE[j]!=0).sum())])

In [0]:
mRMSE_num_c = []

for i in range(16):
    a = []
    for j in range(predictions.shape[0]):
        a.append(table_num_c[i*4+j][2])
        mRMSE_num_c.append(a)

mRMSE_num_c = np.array(mRMSE_num_c).T
mRMSE_num_c = {model_names[i]:mRMSE_num_c[i] for i in range(mRMSE_num_c.shape[0])}
mRMSE_num_c['x'] = range(0,16)

In [0]:
colors = ['coral', 'goldenrod', 'mediumseagreen', 'steelblue']
plt.rcParams["figure.figsize"] = [8.0, 5.0]
for i in range(predictions.shape[0]):
    plt.plot( 'x', model_names[i], data=mRMSE_num_c, marker='o', color=colors[i], linewidth=2, label=model_names[i])
plt.legend()
plt.xticks(mRMSE_num_c['x'])
plt.ylabel('Count Error (mRMSE)')
plt.xlabel('Counts')
plt.grid(True)

In [0]:
count_c = np.zeros((16, len(classes)))
count = np.zeros(16)
for i in range(16):
    for c in range(len(classes)):
        count_c[i, c] += ((gts[gts[:, c] == i, c]) != 0).sum()
    count[i] = ((gts.sum(axis=1) == i) != 0).sum()

print(count_c.mean(axis=1))
print(count)

In [0]:
plt.bar(range(1,16), count_c.mean(axis=1)[1:])
plt.xticks(range(1, 16))
plt.ylabel('Average # of Images per Class per Count')
plt.xlabel('Counts')

In [0]:
plt.bar(range(1,16), count[1:])
plt.xticks(range(1, 16))
plt.ylabel('# of Images per Count')
plt.xlabel('Counts')

In [0]:
under = []
over = []
correct = []

for p in predictions:
    under.append(np.round(p) < gts)
    over.append(np.round(p) > gts)
    correct.append(np.round(p) == gts)

    
under = np.array(under)
over = np.array(over)
correct = np.array(correct)

print(under.shape)

totals = under.sum(axis=1) + over.sum(axis=1)  + correct.sum(axis=1) 

print("Underestimate: ", under.sum(axis=1)  / totals)
print("Overestimate: ", over.sum(axis=1)  / totals)
print("Correct: ", correct.sum(axis=1)  / totals)

In [0]:
percentage = np.zeros((predictions.shape[0], 15, len(classes), 3))
for i in range(1, 16):
    for j, p in enumerate(predictions):
        for c in range(len(classes)):
            mask = gts_totnum == i
            mask2 = np.logical_and(gts[:, c] + np.round(p[:, c]) > 0.0, mask)
            if gts[mask2, c].shape[0] == 0:
                continue
            else:
                total = under[j, mask2, c].sum() + over[j, mask2, c].sum() + correct[j, mask2, c].sum()
                percentage[j, i-1, c, 0] = under[j, mask2, c].sum() / total
                percentage[j, i-1, c, 1] = over[j, mask2,c ].sum() / total
                percentage[j, i-1, c, 2] = correct[j, mask2, c].sum() / total

In [0]:
percentage_total = np.zeros((predictions.shape[0], 15, 3))
for i in range(1, 16):
    for j, p in enumerate(predictions):
        total = 0
        for c in range(len(classes)):
        mask = gts_totnum == i
        mask2 = np.logical_and(gts[:, c] + np.round(p[:, c]) > 0.0, mask)
        if gts[mask2, c].shape[0] == 0:
            continue
        else:
            total += under[j, mask2, c].sum() + over[j, mask2, c].sum() + correct[j, mask2, c].sum()
            percentage_total[j, i-1, 0] += under[j, mask2, c].sum()
            percentage_total[j, i-1, 1] += over[j, mask2,c ].sum()
            percentage_total[j, i-1, 2] += correct[j, mask2, c].sum()
        percentage_total[j, i-1, :] /= total

In [0]:
percentage_mean = percentage.sum(axis=2) / np.expand_dims(percentage.sum(axis=2).sum(axis=2), axis=2)
percentage_mean_global = percentage.sum(axis=1).sum(axis=1) / np.expand_dims(percentage.sum(axis=1).sum(axis=1).sum(axis=1), axis=1)
print(percentage_mean_global.shape)

In [0]:
ax = plt.subplot()
ax.bar(np.array(range(0, 4)) - 0.3, percentage_mean_global[:,2], width=0.3, color='g', align='center', label='correct')
ax.bar(np.array(range(0, 4)), percentage_mean_global[:,0], width=0.3, color='b', align='center', label='under-estimated')
ax.bar(np.array(range(0, 4)) + 0.3, percentage_mean_global[:, 1], width=0.3, color='r', align='center', label='over-estimated')

plt.xticks(range(4), labels=['ssc-basic', 'ssc-m', 'ssc-m-c', 'ssc-m-lstm-c'])
plt.ylabel('Percentage')
plt.xlabel('Models')
plt.legend()
plt.show()
  
for p in percentage_total:
    ax = plt.subplot()
    ax.bar(np.array(range(1, 16)) - 0.3, p[:, 2], width=0.3, color='g', align='center', label='correct')
    ax.bar(np.array(range(1, 16)), p[:, 0], width=0.3, color='b', align='center', label='under-estimated')
    ax.bar(np.array(range(1, 16)) + 0.3, p[:, 1], width=0.3, color='r', align='center', label='over-estimated')

    plt.xticks(range(1, 16))
    plt.ylabel('Percentage')
    plt.xlabel('Counts')
    plt.legend()
    plt.show()

In [0]:
i = random.randint(0, 4952)
# 4469 plane and reflected people
# 2106 bottles

imshow(imgs[i])
print(i)
pretty_prediction_multimodel(classes, predictions[:, i:i+1], gts[i:i+1], i)